In [3]:
import blockcypher as bc
import requests
from datetime import datetime, timezone

In [ ]:
import blockcypher as bc
import requests
from datetime import datetime, timezone

# This is specifically for BTC
# check the api url

# Step 1: Get the latest block hash
block_url = "https://api.blockcypher.com/v1/btc/main"
block_resp = requests.get(block_url).json()
block_hash = block_resp['hash']

# Step 2: Get block details using block hash
block_detail_url = f"https://api.blockcypher.com/v1/btc/main/blocks/{block_hash}?txstart=1&limit=500"
detail_resp = requests.get(block_detail_url).json()

# Basic data
block_id = detail_resp.get('hash')
num_transactions = detail_resp.get('n_tx')
timestamp_unix = detail_resp.get('time')  # ISO8601 format

# Convert ISO8601 to GMT datetime
timestamp_gmt = datetime.fromisoformat(timestamp_unix.replace("Z", "+00:00")).strftime("%Y-%m-%d %H:%M:%S GMT")

# Step 3: Get transactions in the block
tx_hashes = detail_resp.get('txids', [])  # List of transaction hashes
tx_values = []
tx_fees = []

for tx_hash in tx_hashes:
    tx_url = f"https://api.blockcypher.com/v1/btc/main/txs/{tx_hash}"
    tx_resp = requests.get(tx_url).json()

    total = tx_resp.get('total', 0) / 1e8  # Convert from satoshis to BTC
    fee = tx_resp.get('fees', 0) / 1e8     # Also convert to BTC

    tx_values.append(total)
    tx_fees.append(fee)

# Step 4: Calculate required values
largest_transaction = max(tx_values) if tx_values else 0
avg_transaction = sum(tx_values) / len(tx_values) if tx_values else 0
avg_fee = sum(tx_fees) / len(tx_fees) if tx_fees else 0

# Final Output
print(f"Block ID: {block_id}")
print(f"Number of Transactions: {num_transactions}")
print(f"Largest Transaction (BTC): {largest_transaction:.8f}")
print(f"Average Transaction (BTC): {avg_transaction:.8f}")
print(f"Average Fee (BTC): {avg_fee:.8f}")
print(f"Time (GMT): {timestamp_gmt}")


Block ID: 00000000000000000001c031bdebae713407ff051636e809ed9fffb58a6b08f5
Number of Transactions: 1665
Largest Transaction (BTC): 99.99994510
Average Transaction (BTC): 0.34326117
Average Fee (BTC): 0.00000596
Time (GMT): 2025-04-26 22:53:09 GMT


In [2]:
import requests
from datetime import datetime, timezone
import time
import sqlite3

# Target month/year
TARGET_YEAR = 2025
TARGET_MONTH = 3  # March

# BlockCypher Base URL
BASE_URL = "https://api.blockcypher.com/v1/btc/main"

# Step 1: Create (or connect to) the SQLite database
conn = sqlite3.connect('bitcoin_march.db')
cursor = conn.cursor()

# Step 2: Create table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS march_blocks (
    block_id TEXT PRIMARY KEY,
    num_transactions INTEGER,
    largest_transaction REAL,
    avg_transaction REAL,
    avg_fee REAL,
    timestamp_gmt TEXT
)
''')
conn.commit()

# Step 3: Get the latest block hash
resp = requests.get(BASE_URL).json()
current_block_hash = resp['hash']

while True:
    # Step 4: Get block details
    block_detail_url = f"{BASE_URL}/blocks/{current_block_hash}?txstart=1&limit=500"
    detail_resp = requests.get(block_detail_url).json()

    timestamp_iso = detail_resp.get('time')  # ISO8601
    if timestamp_iso is None:
        break  # No timestamp? Something's wrong, exit

    block_time = datetime.fromisoformat(timestamp_iso.replace("Z", "+00:00"))

    # Skip blocks after March
    if (block_time.year > TARGET_YEAR) or (block_time.year == TARGET_YEAR and block_time.month > TARGET_MONTH):
        current_block_hash = detail_resp.get('prev_block')
        if not current_block_hash:
            break
        continue

    # Stop if before March
    if (block_time.year < TARGET_YEAR) or (block_time.year == TARGET_YEAR and block_time.month < TARGET_MONTH):
        break

    # Process block
    block_id = detail_resp.get('hash')
    num_transactions = detail_resp.get('n_tx')
    tx_hashes = detail_resp.get('txids', [])

    tx_values = []
    tx_fees = []

    for tx_hash in tx_hashes:
        tx_url = f"{BASE_URL}/txs/{tx_hash}"
        tx_resp = requests.get(tx_url).json()

        total = tx_resp.get('total', 0) / 1e8
        fee = tx_resp.get('fees', 0) / 1e8

        tx_values.append(total)
        tx_fees.append(fee)

        # Small delay
        time.sleep(0.2)

    largest_transaction = max(tx_values) if tx_values else 0
    avg_transaction = sum(tx_values) / len(tx_values) if tx_values else 0
    avg_fee = sum(tx_fees) / len(tx_fees) if tx_fees else 0

    timestamp_gmt = block_time.strftime("%Y-%m-%d %H:%M:%S GMT")

    # Step 5: Insert into database
    cursor.execute('''
    INSERT OR REPLACE INTO march_blocks (block_id, num_transactions, largest_transaction, avg_transaction, avg_fee, timestamp_gmt)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (block_id, num_transactions, largest_transaction, avg_transaction, avg_fee, timestamp_gmt))
    conn.commit()

    print(f"Inserted Block: {block_id} at {timestamp_gmt}")

    # Move to previous block
    current_block_hash = detail_resp.get('prev_block')
    if not current_block_hash:
        break

    time.sleep(0.5)

# Step 6: Close database connection
conn.close()

print("\nFinished inserting March block data into bitcoin_march.db.")


KeyError: 'hash'

In [3]:
print(resp)

{'error': 'Limits reached.'}
